# Proof of Concept for Final Capstone

### Week 3, Part 1 - Preparing the data

In [3]:
!conda install -c conda-forge beautifulsoup4 --yes

!conda install -c conda-forge geopy --yes

!conda install -c conda-forge folium=0.5.0 --yes

print('Libraries installed!')


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    soupsieve-2.0.1            |   py36h9f0ad1d_0          56 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         219 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.1-py36h9f0ad1d_0
  soupsieve          conda-forge/linux-64::soupsieve-2.0.1-py36h9f0ad1d_0



beautifulsoup4-4.9.1 | 163 KB    | ##################################### | 100% 
soupsieve-2.0.1      | 56 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction

In [4]:
# Import Libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [5]:
df = pd.read_csv('Pune-data.csv', delimiter=';')
df.head()

country code  postal code    place name  admin name1  admin code1  \
0           IN       410403  Kaivalyadham  Maharashtra           16   
1           IN       410405     Takave KH  Maharashtra           16   
2           IN       410406       Shivali  Maharashtra           16   
3           IN       410502         Kusur  Maharashtra           16   
4           IN       410502         Godre  Maharashtra           16   

  admin name2  admin code2 admin name3  admin code3  latitude  longitude  \
0        Pune          521       Maval          NaN   19.4781    73.7845   
1        Pune          521       Maval          NaN   19.6594    73.8791   
2        Pune          521       Maval          NaN   19.4781    73.7845   
3        Pune          521      Junnar          NaN   18.5709    74.0691   
4        Pune          521      Junnar          NaN   19.3864    74.4703   

   accuracy      coordinates  
0         1  19.4781,73.7845  
1         3  19.6594,73.8791  
2         1  19.4781,73.7845  
3         4  18.5709,74.0691  
4         3  19.3864,74.4703

In [6]:
df.replace("?", np.nan, inplace = True)
df.head(5)

country code  postal code    place name  admin name1  admin code1  \
0           IN       410403  Kaivalyadham  Maharashtra           16   
1           IN       410405     Takave KH  Maharashtra           16   
2           IN       410406       Shivali  Maharashtra           16   
3           IN       410502         Kusur  Maharashtra           16   
4           IN       410502         Godre  Maharashtra           16   

  admin name2  admin code2 admin name3  admin code3  latitude  longitude  \
0        Pune          521       Maval          NaN   19.4781    73.7845   
1        Pune          521       Maval          NaN   19.6594    73.8791   
2        Pune          521       Maval          NaN   19.4781    73.7845   
3        Pune          521      Junnar          NaN   18.5709    74.0691   
4        Pune          521      Junnar          NaN   19.3864    74.4703   

   accuracy      coordinates  
0         1  19.4781,73.7845  
1         3  19.6594,73.8791  
2         1  19.4781,73.7845  
3         4  18.5709,74.0691  
4         3  19.3864,74.4703

In [7]:
missing_data = df.isnull()
missing_data.head(12)

country code  postal code  place name  admin name1  admin code1  \
0          False        False       False        False        False   
1          False        False       False        False        False   
2          False        False       False        False        False   
3          False        False       False        False        False   
4          False        False       False        False        False   
5          False        False       False        False        False   
6          False        False       False        False        False   
7          False        False       False        False        False   
8          False        False       False        False        False   
9          False        False       False        False        False   
10         False        False       False        False        False   
11         False        False       False        False        False   

    admin name2  admin code2  admin name3  admin code3  latitude  longitude  \
0         False        False        False         True     False      False   
1         False        False        False         True     False      False   
2         False        False        False         True     False      False   
3         False        False        False         True     False      False   
4         False        False        False         True     False      False   
5         False        False        False         True     False      False   
6         False        False        False         True     False      False   
7         False        False        False         True     False      False   
8         False        False        False         True     False      False   
9         False        False        False         True     False      False   
10        False        False        False         True     False      False   
11        False        False        False         True     False      False   

    accuracy  coordinates  
0      False        False  
1      False        False  
2      False        False  
3      False        False  
4      False        False  
5      False        False  
6      False        False  
7      False        False  
8      False        False  
9      False        False  
10     False        False  
11     False        False

In [8]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")    

country code
False    791
Name: country code, dtype: int64

postal code
False    791
Name: postal code, dtype: int64

place name
False    791
Name: place name, dtype: int64

admin name1
False    791
Name: admin name1, dtype: int64

admin code1
False    791
Name: admin code1, dtype: int64

admin name2
False    791
Name: admin name2, dtype: int64

admin code2
False    791
Name: admin code2, dtype: int64

admin name3
False    789
True       2
Name: admin name3, dtype: int64

admin code3
True    791
Name: admin code3, dtype: int64

latitude
False    791
Name: latitude, dtype: int64

longitude
False    791
Name: longitude, dtype: int64

accuracy
False    791
Name: accuracy, dtype: int64

coordinates
False    791
Name: coordinates, dtype: int64



In [9]:
df.shape

(791, 13)

In [10]:
df.head(5)

country code  postal code    place name  admin name1  admin code1  \
0           IN       410403  Kaivalyadham  Maharashtra           16   
1           IN       410405     Takave KH  Maharashtra           16   
2           IN       410406       Shivali  Maharashtra           16   
3           IN       410502         Kusur  Maharashtra           16   
4           IN       410502         Godre  Maharashtra           16   

  admin name2  admin code2 admin name3  admin code3  latitude  longitude  \
0        Pune          521       Maval          NaN   19.4781    73.7845   
1        Pune          521       Maval          NaN   19.6594    73.8791   
2        Pune          521       Maval          NaN   19.4781    73.7845   
3        Pune          521      Junnar          NaN   18.5709    74.0691   
4        Pune          521      Junnar          NaN   19.3864    74.4703   

   accuracy      coordinates  
0         1  19.4781,73.7845  
1         3  19.6594,73.8791  
2         1  19.4781,73.7845  
3         4  18.5709,74.0691  
4         3  19.3864,74.4703

In [11]:
df_final = df[['country code', 'admin name1', 'admin name2', 'admin name3', 'place name', 'postal code', 'latitude' ,'longitude']]
df_final.head()


country code  admin name1 admin name2 admin name3    place name  \
0           IN  Maharashtra        Pune       Maval  Kaivalyadham   
1           IN  Maharashtra        Pune       Maval     Takave KH   
2           IN  Maharashtra        Pune       Maval       Shivali   
3           IN  Maharashtra        Pune      Junnar         Kusur   
4           IN  Maharashtra        Pune      Junnar         Godre   

   postal code  latitude  longitude  
0       410403   19.4781    73.7845  
1       410405   19.6594    73.8791  
2       410406   19.4781    73.7845  
3       410502   18.5709    74.0691  
4       410502   19.3864    74.4703

In [11]:
df_final.shape

(791, 8)

In [18]:
df_final.rename(columns={"country code": "Country", "admin name1":"State", "admin name2":"City", "admin name3":"Neighbourhood","place name":"Place", "postal code":"Postalcode"}, inplace=True)
df_final.shape
df_final.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Country        State  City Neighbourhood         Place  Postalcode  \
0      IN  Maharashtra  Pune         Maval  Kaivalyadham      410403   
1      IN  Maharashtra  Pune         Maval     Takave KH      410405   
2      IN  Maharashtra  Pune         Maval       Shivali      410406   
3      IN  Maharashtra  Pune        Junnar         Kusur      410502   
4      IN  Maharashtra  Pune        Junnar         Godre      410502   

   latitude  longitude  
0   19.4781    73.7845  
1   19.6594    73.8791  
2   19.4781    73.7845  
3   18.5709    74.0691  
4   19.3864    74.4703

In [19]:
df_final.shape

(791, 8)

In [20]:
# Get the latitude and longitudinal values for Pune. Use geopy library
address = 'Pune'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Pune are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Pune are 18.521428, 73.8544541.


In [15]:
# create map of Toronto using latitude and longitude values
map_pune = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, place, neighbourhood in zip(df_final['latitude'], df_final['longitude'], df_final['Place'], df_final['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, place)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_pune)  
    
map_pune

In [21]:
df_final.replace("?", np.nan)
df_final.dropna(inplace=True)
neighbourhood_names = list(df_final.Neighbourhood.unique())
neighbourhood_with_pune_city = []
neighbourhood_names

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


['Maval',
 'Junnar',
 'Ambegaon',
 'Khed',
 'Pune City',
 'Haveli',
 'Mulashi',
 'Purandar',
 'Velhe',
 'Mulshi',
 'Daund',
 'Bhor',
 'Shirur',
 'Baramati',
 'Indapur',
 'Shiurur',
 'Mawal',
 'Pune',
 'Purandhar']

In [22]:
neighbourhood_names = list(df_final.Neighbourhood.unique())
neighbourhood_with_pune_city = []

for x in neighbourhood_names:
   
    if "pune" in x.lower():
        neighbourhood_with_pune_city.append(x)
        
neighbourhood_with_pune_city

['Pune City', 'Pune']

In [23]:
# create a new DataFrame with only boroughs that contain the word Toronto
df_pune = df_final[df_final['Neighbourhood'].isin(neighbourhood_with_pune_city)].reset_index(drop=True)
df_pune

Country        State  City Neighbourhood                      Place  \
0        IN  Maharashtra  Pune     Pune City             Pune New Bazar   
1        IN  Maharashtra  Pune     Pune City      Shukrawar Peth (Pune)   
2        IN  Maharashtra  Pune     Pune City           Khadakwasla R.S.   
3        IN  Maharashtra  Pune     Pune City               Narayan Peth   
4        IN  Maharashtra  Pune     Pune City                Bhosarigoan   
5        IN  Maharashtra  Pune     Pune City             Vadgaon Budruk   
6        IN  Maharashtra  Pune     Pune City                 Kondhwa BK   
7        IN  Maharashtra  Pune     Pune City               Raviwar Peth   
8        IN  Maharashtra  Pune     Pune City                 Baner Road   
9        IN  Maharashtra  Pune     Pune City    Shivaji Housing Society   
10       IN  Maharashtra  Pune     Pune City                   Armament   
11       IN  Maharashtra  Pune     Pune City      Ex. Serviceman Colony   
12       IN  Maharashtra  Pune     Pune City               Bhavani Peth   
13       IN  Maharashtra  Pune     Pune City                     N.I.A.   
14       IN  Maharashtra  Pune     Pune City             Vadgaon Shinde   
15       IN  Maharashtra  Pune     Pune City                Mohamadwadi   
16       IN  Maharashtra  Pune     Pune City                   Pune H.O   
17       IN  Maharashtra  Pune     Pune City             Ghorpuri Bazar   
18       IN  Maharashtra  Pune     Pune City               Khadki Bazar   
19       IN  Maharashtra  Pune     Pune City             Film Institute   
20       IN  Maharashtra  Pune     Pune City          Govt. Polytechnic   
21       IN  Maharashtra  Pune     Pune City              Pimpri Colony   
22       IN  Maharashtra  Pune     Pune City                   Hadapsar   
23       IN  Maharashtra  Pune     Pune City                Iaf Station   
24       IN  Maharashtra  Pune     Pune City             Salisbury Park   
25       IN  Maharashtra  Pune     Pune City               Guruwar Peth   
26       IN  Maharashtra  Pune     Pune City                 Kondhwa Lh   
27       IN  Maharashtra  Pune     Pune City            Pune Cantt East   
28       IN  Maharashtra  Pune     Pune City            Deccan Gymkhana   
29       IN  Maharashtra  Pune     Pune City           S.S.C.Exam Board   
30       IN  Maharashtra  Pune     Pune City        Congress House Road   
31       IN  Maharashtra  Pune     Pune City    Botanical Garden (Pune)   
32       IN  Maharashtra  Pune     Pune City         N.D.A. Khadakwasla   
33       IN  Maharashtra  Pune     Pune City                    Sangavi   
34       IN  Maharashtra  Pune     Pune City                      C M E   
35       IN  Maharashtra  Pune     Pune City                 Dhankawadi   
36       IN  Maharashtra  Pune     Pune City          Anandnagar (Pune)   
37       IN  Maharashtra  Pune     Pune City                 Karvenagar   
38       IN  Maharashtra  Pune     Pune City              Dr.B.A. Chowk   
39       IN  Maharashtra  Pune     Pune City               Parvati Gaon   
40       IN  Maharashtra  Pune     Pune City             Chinchwad East   
41       IN  Maharashtra  Pune     Pune City             Airport (Pune)   
42       IN  Maharashtra  Pune     Pune City                  Kasarwadi   
43       IN  Maharashtra  Pune     Pune City         Market Yard (Pune)   
44       IN  Maharashtra  Pune     Pune City             Swargate Chowk   
45       IN  Maharashtra  Pune     Pune City                      Wakad   
46       IN  Maharashtra  Pune     Pune City               N.W. College   
47       IN  Maharashtra  Pune     Pune City                  Nana Peth   
48       IN  Maharashtra  Pune     Pune City                 Aundh T.S.   
49       IN  Maharashtra  Pune     Pune City                    Parvati   
50       IN  Maharashtra  Pune          Pune                Viman nagar   
51       IN  Maharashtra  Pune     Pune City                       Srpf   
52       IN  Maharashtra  Pun

In [24]:
# Onto Foursquare now - define Foursquare Credentials and Version
CLIENT_ID = 'XLWPZDPXCTB454CSEBAN3NDHF0FQZK2MWETDQXY4YJX4Y0AL' # your Foursquare ID
CLIENT_SECRET = 'MUPJR0JXTIEJQJSAFLARJYTWBYKBOTFGTFKTQRKNDT2V5H2U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XLWPZDPXCTB454CSEBAN3NDHF0FQZK2MWETDQXY4YJX4Y0AL
CLIENT_SECRET:MUPJR0JXTIEJQJSAFLARJYTWBYKBOTFGTFKTQRKNDT2V5H2U


In [25]:
# Get the top 100 venues that are within a radius of 500 meters.

radius = 500
LIMIT = 100

venues = []

for lat, long, post, place, neighbourhood in zip(df_pune['latitude'], df_pune['longitude'], df_pune['Postalcode'], df_pune['Place'], df_pune['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            place,
            neighbourhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [27]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Place', 'Neighborhood', 'PlaceLatitude', 'PlaceLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(560, 9)


PostalCode           Place Neighborhood  PlaceLatitude  PlaceLongitude  \
0      411001  Pune New Bazar    Pune City        18.5372         73.8551   
1      411001  Pune New Bazar    Pune City        18.5372         73.8551   
2      411001  Pune New Bazar    Pune City        18.5372         73.8551   
3      411001  Pune New Bazar    Pune City        18.5372         73.8551   
4      411001  Pune New Bazar    Pune City        18.5372         73.8551   

                    VenueName  VenueLatitude  VenueLongitude     VenueCategory  
0              Shopper's Stop      18.534332       73.851900  Department Store  
1  Velocita Brand Consultants      18.536705       73.859303       IT Services  
2            Bajaj Brand View      18.538656       73.850995   Motorcycle Shop  
3           Suruchi Fast food      18.538394       73.850650       Snack Place  
4                Jason Martin      18.538231       73.850591      Intersection

In [28]:
# Check how many venues were returned for each PostalCode

venues_df.groupby(["PostalCode", "Place", "Neighborhood"]).count()

PlaceLatitude  \
PostalCode Place                     Neighborhood                  
411001     C D A (O)                 Pune City                 5   
           Dr.B.A. Chowk             Pune City                 5   
           Ghorpuri Bazar            Pune City                 5   
           N.W. College              Pune City                 5   
           Pune Cantt East           Pune City                 5   
           Pune H.O                  Pune City                 5   
           Pune New Bazar            Pune City                 5   
           Sachapir Street           Pune City                 5   
411002     Bajirao Road              Pune City                 5   
           Nana Peth                 Pune City                 5   
           Pune City H.O             Pune City                 5   
           Raviwar Peth              Pune City                 5   
           Shukrawar Peth (Pune)     Pune City                 5   
411003     Ammunition Factory Khadki Pune City                 5   
           H.E. Factory              Pune City                 5   
           Khadki                    Pune City                 5   
           Khadki Bazar              Pune City                 5   
411004     A.R. Shala                Pune City                14   
           Deccan Gymkhana           Pune City                14   
           Film Institute            Pune City                14   
411005     Congress House Road       Pune City                20   
           S.S.C.Exam Board          Pune City                20   
           Shivajinagar (Pune)       Pune City                20   
411006     Yerwada                   Pune City                 3   
411007     Aundh T.S.                Pune City                 4   
           Ganeshkhind               Pune City                 4   
411008     Baner Road                Pune City                 5   
           N.C.L. Pune               Pune City                 5   
411009     Parvati                   Pune City                 5   
           Parvati Gaon              Pune City                 5   
411011     Kasba Peth                Pune City                 5   
           Mangalwar Peth (Pune)     Pune City                 5   
           Rasta Peth                Pune City                 5   
411012     Dapodi                    Pune City                 3   
           Dapodi Bazar              Pune City                 3   
411013     Hadpsar I.E.              Pune City                 5   
411014     9 DRD                     Pune City                36   
           Dukirkline                Pune City                36   
           Vadgaon Sheri             Pune City                36   
           Viman nagar               Pune                     36   
411015     Dighi Camp                Pune City                 5   
411016     Govt. Polytechnic         Pune City                 5   
           Model Colony              Pune City                 5   
           Shivaji Housing Society   Pune City                 5   
411017     Kalewadi                  Pune City                 5   
           Pimpri Colony             Pune City                 5   
           Pimpri Waghire            Pune City                 5   
411018     Masulkar Colony           Pune City                 5   
           Pimpri P F                Pune City                 5   
411019     Chinchwad East            Pune City                 5   
411021     Armament                  Pune City                12   
411030     Lokmanyanagar             Pune City                 4   
           Narayan Peth              Pune City                 4   
           Rashtra Bhasha Bhavan     Pune City                 4   
           S.P. College              Pune City                 4   
           Sadashiv Peth             Pune City                 4   
           Shaniwar Peth (Pune)      Pune City                 4   
411032     Airport (Pune)            Pu

In [29]:
# Analysis of each area
# one hot encoding
pune_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
pune_onehot['PostalCode'] = venues_df['PostalCode'] 
pune_onehot['Place'] = venues_df['Place'] 
pune_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(pune_onehot.columns[-3:]) + list(pune_onehot.columns[:-3])
pune_onehot = pune_onehot[fixed_columns]

print(pune_onehot.shape)
pune_onehot.head()

(560, 49)


PostalCode           Place Neighborhoods  Bakery  Bistro  Breakfast Spot  \
0      411001  Pune New Bazar     Pune City       0       0               0   
1      411001  Pune New Bazar     Pune City       0       0               0   
2      411001  Pune New Bazar     Pune City       0       0               0   
3      411001  Pune New Bazar     Pune City       0       0               0   
4      411001  Pune New Bazar     Pune City       0       0               0   

   Burger Joint  Café  Cheese Shop  Chinese Restaurant  Coffee Shop  \
0             0     0            0                   0            0   
1             0     0            0                   0            0   
2             0     0            0                   0            0   
3             0     0            0                   0            0   
4             0     0            0                   0            0   

   Deli / Bodega  Department Store  Dessert Shop  Electronics Store  \
0              0                 1             0                  0   
1              0                 0             0                  0   
2              0                 0             0                  0   
3              0                 0             0                  0   
4              0                 0             0                  0   

   Fast Food Restaurant  Fried Chicken Joint  Garden  Gym  \
0                     0                    0       0    0   
1                     0                    0       0    0   
2                     0                    0       0    0   
3                     0                    0       0    0   
4                     0                    0       0    0   

   Gym / Fitness Center  Hotel  Hotel Bar  IT Services  Ice Cream Shop  \
0                     0      0          0            0               0   
1                     0      0          0            1               0   
2                     0      0          0            0               0   
3                     0      0          0            0               0   
4                     0      0          0            0               0   

   Indian Restaurant  Intersection  Jewelry Store  Juice Bar  Lawyer  Lounge  \
0                  0             0              0          0       0       0   
1                  0             0              0          0       0       0   
2                  0             0              0          0       0       0   
3                  0             0              0          0       0       0   
4                  0             1              0          0       0       0   

   Maharashtrian Restaurant  Market  Mobile Phone Shop  Motorcycle Shop  \
0                         0       0                  0                0   
1                         0       0                  0                0   
2                         0       0                  0                1   
3                         0       0                  0                0   
4                         0       0                  0                0   

   Nature Preserve  Outlet Mall  Park  Pizza Place  Restaurant  \
0                0            0     0            0           0   
1                0            0     0            0           0   
2                0            0     0            0           0   
3                0            0     0            0           0   
4                0            0     0            0           0   

   Seafood Restaurant  Shop & Service  Shopping Mall  Snack Place  \
0                   0               0              0            0   
1                   0               0              0            0   
2                   0               0              0            0   
3                   0               0              0            1   
4                   0               0              0            0   

   Soccer Field  South Indian Restaurant  Southern / Soul Food Restaurant  \
0             0                        0 

In [30]:
# Now group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

pune_grouped = pune_onehot.groupby(["PostalCode", "Place", "Neighborhoods"]).mean().reset_index()

print(pune_grouped.shape)
pune_grouped

(80, 49)


PostalCode                      Place Neighborhoods    Bakery    Bistro  \
0       411001                  C D A (O)     Pune City  0.000000  0.000000   
1       411001              Dr.B.A. Chowk     Pune City  0.000000  0.000000   
2       411001             Ghorpuri Bazar     Pune City  0.000000  0.000000   
3       411001               N.W. College     Pune City  0.000000  0.000000   
4       411001            Pune Cantt East     Pune City  0.000000  0.000000   
5       411001                   Pune H.O     Pune City  0.000000  0.000000   
6       411001             Pune New Bazar     Pune City  0.000000  0.000000   
7       411001            Sachapir Street     Pune City  0.000000  0.000000   
8       411002               Bajirao Road     Pune City  0.000000  0.000000   
9       411002                  Nana Peth     Pune City  0.000000  0.000000   
10      411002              Pune City H.O     Pune City  0.000000  0.000000   
11      411002               Raviwar Peth     Pune City  0.000000  0.000000   
12      411002      Shukrawar Peth (Pune)     Pune City  0.000000  0.000000   
13      411003  Ammunition Factory Khadki     Pune City  0.200000  0.000000   
14      411003               H.E. Factory     Pune City  0.200000  0.000000   
15      411003                     Khadki     Pune City  0.200000  0.000000   
16      411003               Khadki Bazar     Pune City  0.200000  0.000000   
17      411004                 A.R. Shala     Pune City  0.000000  0.071429   
18      411004            Deccan Gymkhana     Pune City  0.000000  0.071429   
19      411004             Film Institute     Pune City  0.000000  0.071429   
20      411005        Congress House Road     Pune City  0.050000  0.000000   
21      411005           S.S.C.Exam Board     Pune City  0.050000  0.000000   
22      411005        Shivajinagar (Pune)     Pune City  0.050000  0.000000   
23      411006                    Yerwada     Pune City  0.000000  0.000000   
24      411007                 Aundh T.S.     Pune City  0.000000  0.000000   
25      411007                Ganeshkhind     Pune City  0.000000  0.000000   
26      411008                 Baner Road     Pune City  0.000000  0.000000   
27      411008                N.C.L. Pune     Pune City  0.000000  0.000000   
28      411009                    Parvati     Pune City  0.000000  0.000000   
29      411009               Parvati Gaon     Pune City  0.000000  0.000000   
30      411011                 Kasba Peth     Pune City  0.000000  0.000000   
31      411011      Mangalwar Peth (Pune)     Pune City  0.000000  0.000000   
32      411011                 Rasta Peth     Pune City  0.000000  0.000000   
33      411012                     Dapodi     Pune City  0.000000  0.000000   
34      411012               Dapodi Bazar     Pune City  0.000000  0.000000   
35      411013               Hadpsar I.E.     Pune City  0.000000  0.000000   
36      411014                      9 DRD     Pune City  0.027778  0.027778   
37      411014                 Dukirkline     Pune City  0.027778  0.027778   
38      411014              Vadgaon Sheri     Pune City  0.027778  0.027778   
39      411014                Viman nagar          Pune  0.027778  0.027778   
40      411015                 Dighi Camp     Pune City  0.000000  0.000000   
41      411016          Govt. Polytechnic     Pune City  0.000000  0.000000   
42      411016               Model Colony     Pune City  0.000000  0.000000   
43      411016    Shivaji Housing Society     Pune City  0.000000  0.000000   
44      411017                   Kalewadi     Pune City  0.000000  0.000000   
45      411017              Pimpri Colony     Pune City  0.000000  0.000000   
46      411017             Pimpri Waghire     Pune City  0.000000  0.000000   
47      411018            Masulkar Colony     Pune City  0.000000  0.000000   
48      411018                 Pimpri P F     Pune City  0.000000  0.000000   
49      411019             Chinchwad East     Pune Cit

In [31]:
# Display the top 10 venues for each PostalCode in a new dataframe.


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Place', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = pune_grouped['PostalCode']
neighborhoods_venues_sorted['Place'] = pune_grouped['Place']
neighborhoods_venues_sorted['Neighborhoods'] = pune_grouped['Neighborhoods']

for ind in np.arange(pune_grouped.shape[0]):
    row_categories = pune_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(80, 13)


PostalCode                      Place Neighborhoods 1st Most Common Venue  \
0       411001                  C D A (O)     Pune City          Intersection   
1       411001              Dr.B.A. Chowk     Pune City          Intersection   
2       411001             Ghorpuri Bazar     Pune City          Intersection   
3       411001               N.W. College     Pune City          Intersection   
4       411001            Pune Cantt East     Pune City          Intersection   
5       411001                   Pune H.O     Pune City          Intersection   
6       411001             Pune New Bazar     Pune City          Intersection   
7       411001            Sachapir Street     Pune City          Intersection   
8       411002               Bajirao Road     Pune City          Intersection   
9       411002                  Nana Peth     Pune City          Intersection   
10      411002              Pune City H.O     Pune City          Intersection   
11      411002               Raviwar Peth     Pune City          Intersection   
12      411002      Shukrawar Peth (Pune)     Pune City          Intersection   
13      411003  Ammunition Factory Khadki     Pune City      Department Store   
14      411003               H.E. Factory     Pune City      Department Store   
15      411003                     Khadki     Pune City      Department Store   
16      411003               Khadki Bazar     Pune City      Department Store   
17      411004                 A.R. Shala     Pune City           Snack Place   
18      411004            Deccan Gymkhana     Pune City           Snack Place   
19      411004             Film Institute     Pune City           Snack Place   
20      411005        Congress House Road     Pune City        Ice Cream Shop   
21      411005           S.S.C.Exam Board     Pune City        Ice Cream Shop   
22      411005        Shivajinagar (Pune)     Pune City        Ice Cream Shop   
23      411006                    Yerwada     Pune City    Chinese Restaurant   
24      411007                 Aundh T.S.     Pune City            Restaurant   
25      411007                Ganeshkhind     Pune City            Restaurant   
26      411008                 Baner Road     Pune City          Intersection   
27      411008                N.C.L. Pune     Pune City          Intersection   
28      411009                    Parvati     Pune City          Intersection   
29      411009               Parvati Gaon     Pune City          Intersection   
30      411011                 Kasba Peth     Pune City     Indian Restaurant   
31      411011      Mangalwar Peth (Pune)     Pune City     Indian Restaurant   
32      411011                 Rasta Peth     Pune City     Indian Restaurant   
33      411012                     Dapodi     Pune City             Hotel Bar   
34      411012               Dapodi Bazar     Pune City             Hotel Bar   
35      411013               Hadpsar I.E.     Pune City     Indian Restaurant   
36      411014                      9 DRD     Pune City     Indian Restaurant   
37      411014                 Dukirkline     Pune City     Indian Restaurant   
38      411014              Vadgaon Sheri     Pune City     Indian Restaurant   
39      411014                Viman nagar          Pune     Indian Restaurant   
40      411015                 Dighi Camp     Pune City         Shopping Mall   
41      411016          Govt. Polytechnic     Pune City     Indian Restaurant   
42      411016               Model Colony     Pune City     Indian Restaurant   
43      411016    Shivaji Housing Society     Pune City     Indian Restaurant   
44      411017                   Kalewadi     Pune City     Indian Restaurant   
45      411017              Pimpri Colony     Pune City     Indian Restaurant   
46      411017             Pimpri Waghire     Pune City     Indian Restaurant   
47      411018            Masulkar Colony     Pune City     Indian Restaurant   
48      411018                 Pimp

In [32]:
# CLuster Pune areas into 5

# set number of clusters
kclusters = 5

pune_grouped_clustering = pune_grouped.drop(["PostalCode", "Place", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pune_grouped_clustering)

# check cluster labels generated for each row in the dataframe
# kmeans.labels_[0:114]
# kmeans.labels_.shape
# pune_grouped.shape
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [106]:
pune_lat = df_pune[["Postalcode", "latitude", "longitude", "Place"]].copy()
pune_lat.rename(columns={"Postalcode": "PostalCode"}, inplace=True)



# pune_lat.reindex(columns=['PostalCode','Place','latitude', 'longitude'])
# pune_lat_grouped=pune_lat.groupby(["PostalCode", "Place"]).mean()
# pune_lat_grouped
# pune_tst


# pune_tst = neighborhoods_venues_sorted.copy()

res = pd.merge(pune_tst, pune_lat, on=['PostalCode', 'Place'], how='left', validate="one_to_many")

d1 = res.pop('latitude')
d2 = res.pop('longitude')
res.insert(3, "latitude", d1, True)
res.insert(4, "longitude", d2, True)

res

# pune_lat.shape

#for column in pune_tst.columns.values.tolist():
#    print(column)
#    print (pune_tst[column].value_counts())
#    print("") 

# for i in range(len(pune_lat)) : 
  # print(i, pune_tst.loc[i, "PostalCode"], pune_tst.loc[i, "Place"]) 
   # pos =  pune_tst.loc[i, "PostalCode"]
   # pl =  pune_tst.loc[i, "Place"]
  # print("--")
  # print((pune_lat.loc["PostalCode"] == pos) & (pune_lat.loc["PostalCode"] == pl))  

# pune_tst# pune_tst
# main_df['header_2'] = main_df['header_1'].map(lookup_df.set_index('header_1')['header_2'])
# pune_tst['Latitude'] = pune_tst['PostalCode'].map(pune_lat.set_index('PostalCode')['PostalCode'])
# out = (pune_tst.merge(pune_lat, left_on='PostalCode, Place', right_on='PostalCode, Place').reindex(columns=['Postalcode', 'latitude', 'longitude', 'Place']))
# out
# pune_tst

PostalCode                      Place Neighborhoods  latitude  longitude  \
0       411001                  C D A (O)     Pune City   18.5372    73.8551   
1       411001              Dr.B.A. Chowk     Pune City   18.5372    73.8551   
2       411001             Ghorpuri Bazar     Pune City   18.5372    73.8551   
3       411001               N.W. College     Pune City   18.5372    73.8551   
4       411001            Pune Cantt East     Pune City   18.5372    73.8551   
5       411001                   Pune H.O     Pune City   18.5372    73.8551   
6       411001             Pune New Bazar     Pune City   18.5372    73.8551   
7       411001            Sachapir Street     Pune City   18.5372    73.8551   
8       411002               Bajirao Road     Pune City   18.5372    73.8551   
9       411002                  Nana Peth     Pune City   18.5372    73.8551   
10      411002              Pune City H.O     Pune City   18.5372    73.8551   
11      411002               Raviwar Peth     Pune City   18.5372    73.8551   
12      411002      Shukrawar Peth (Pune)     Pune City   18.5372    73.8551   
13      411003  Ammunition Factory Khadki     Pune City   18.5635    73.8520   
14      411003               H.E. Factory     Pune City   18.5635    73.8520   
15      411003                     Khadki     Pune City   18.5635    73.8520   
16      411003               Khadki Bazar     Pune City   18.5635    73.8520   
17      411004                 A.R. Shala     Pune City   18.5148    73.8358   
18      411004            Deccan Gymkhana     Pune City   18.5148    73.8358   
19      411004             Film Institute     Pune City   18.5148    73.8358   
20      411005        Congress House Road     Pune City   18.5247    73.8471   
21      411005           S.S.C.Exam Board     Pune City   18.5247    73.8471   
22      411005        Shivajinagar (Pune)     Pune City   18.5247    73.8471   
23      411006                    Yerwada     Pune City   18.5459    73.8855   
24      411007                 Aundh T.S.     Pune City   18.5333    73.8667   
25      411007                Ganeshkhind     Pune City   18.5333    73.8667   
26      411008                 Baner Road     Pune City   18.5372    73.8551   
27      411008                N.C.L. Pune     Pune City   18.5372    73.8551   
28      411009                    Parvati     Pune City   18.5372    73.8551   
29      411009               Parvati Gaon     Pune City   18.5372    73.8551   
30      411011                 Kasba Peth     Pune City   18.5732    73.8739   
31      411011      Mangalwar Peth (Pune)     Pune City   18.5732    73.8739   
32      411011                 Rasta Peth     Pune City   18.5732    73.8739   
33      411012                     Dapodi     Pune City   18.5788    73.8263   
34      411012               Dapodi Bazar     Pune City   18.5788    73.8263   
35      411013               Hadpsar I.E.     Pune City   18.5732    73.8739   
36      411014                      9 DRD     Pune City   18.5685    73.9158   
37      411014                 Dukirkline     Pune City   18.5685    73.9158   
38      411014              Vadgaon Sheri     Pune City   18.5685    73.9158   
39      411014                Viman nagar          Pune   18.5685    73.9158   
40      411015                 Dighi Camp     Pune City   18.5722    73.8796   
41      411016          Govt. Polytechnic     Pune City   18.5732    73.8739   
42      411016               Model Colony     Pune City   18.5732    73.8739   
43      411016    Shivaji Housing Society     Pune City   18.5732    73.8739   
44      411017                   Kalewadi     Pune City   18.5732    73.8739   
45      411017              Pimpri Colony     Pune City   18.5732    73.8739   
46      411017             Pimpri Waghire     Pune City   18.5732    73.8739   
47      411018            Masulkar Colony     Pune City   18.5732    73.8739   
48      411018                 Pimpri P F     Pune City   18.5732    73.8739   
49  

In [33]:
# pune_merged = pune_grouped.copy()
# pune_merged
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
pune_clusters = kmeans.labels_

pune_merged = neighborhoods_venues_sorted.copy()

# get the latitudes & longitudes in place
pune_lat = df_pune[["Postalcode", "latitude", "longitude", "Place"]].copy()
pune_lat.rename(columns={"Postalcode": "PostalCode"}, inplace=True)

pune_final = pd.merge(pune_merged, pune_lat, on=['PostalCode', 'Place'], how='left', validate="one_to_many")

lat = pune_final.pop('latitude')
lon = pune_final.pop('longitude')
pune_final.insert(3, "latitude", lat, True)
pune_final.insert(4, "longitude", lon, True)

# pune_merged
pune_final.insert(5, "Cluster Labels", pune_clusters, True)
pune_final.isnull()
# add clustering labels
# pune_merged["Cluster Labels"] = kmeans.labels_

PostalCode  Place  Neighborhoods  latitude  longitude  Cluster Labels  \
0        False  False          False     False      False           False   
1        False  False          False     False      False           False   
2        False  False          False     False      False           False   
3        False  False          False     False      False           False   
4        False  False          False     False      False           False   
5        False  False          False     False      False           False   
6        False  False          False     False      False           False   
7        False  False          False     False      False           False   
8        False  False          False     False      False           False   
9        False  False          False     False      False           False   
10       False  False          False     False      False           False   
11       False  False          False     False      False           False   
12       False  False          False     False      False           False   
13       False  False          False     False      False           False   
14       False  False          False     False      False           False   
15       False  False          False     False      False           False   
16       False  False          False     False      False           False   
17       False  False          False     False      False           False   
18       False  False          False     False      False           False   
19       False  False          False     False      False           False   
20       False  False          False     False      False           False   
21       False  False          False     False      False           False   
22       False  False          False     False      False           False   
23       False  False          False     False      False           False   
24       False  False          False     False      False           False   
25       False  False          False     False      False           False   
26       False  False          False     False      False           False   
27       False  False          False     False      False           False   
28       False  False          False     False      False           False   
29       False  False          False     False      False           False   
30       False  False          False     False      False           False   
31       False  False          False     False      False           False   
32       False  False          False     False      False           False   
33       False  False          False     False      False           False   
34       False  False          False     False      False           False   
35       False  False          False     False      False           False   
36       False  False          False     False      False           False   
37       False  False          False     False      False           False   
38       False  False          False     False      False           False   
39       False  False          False     False      False           False   
40       False  False          False     False      False           False   
41       False  False          False     False      False           False   
42       False  False          False     False      False           False   
43       False  False          False     False      False           False   
44       False  False          False     False      False           False   
45       False  False          False     False      False           False   
46       False  False          False     False      False           False   
47       False  False          False     False      False           False   
48       False  False          False     False      False           False   
49       False  False          False     False      False           False   
50       False  False          False     False      False           False   


In [34]:
# sort the results by Cluster Labels
print(pune_final.shape)
pune_final.sort_values(["Cluster Labels"], inplace=True)
pune_final.head()

(80, 16)


PostalCode                  Place Neighborhoods  latitude  longitude  \
39      411014            Viman nagar          Pune   18.5685    73.9158   
47      411018        Masulkar Colony     Pune City   18.5732    73.8739   
30      411011             Kasba Peth     Pune City   18.5732    73.8739   
31      411011  Mangalwar Peth (Pune)     Pune City   18.5732    73.8739   
32      411011             Rasta Peth     Pune City   18.5732    73.8739   

    Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
39               0     Indian Restaurant           Coffee Shop   
47               0     Indian Restaurant           Snack Place   
30               0     Indian Restaurant           Snack Place   
31               0     Indian Restaurant           Snack Place   
32               0     Indian Restaurant           Snack Place   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
39    Chinese Restaurant        Ice Cream Shop  Fast Food Restaurant   
47         Shopping Mall                   Gym           Pizza Place   
30         Shopping Mall                   Gym           Pizza Place   
31         Shopping Mall                   Gym           Pizza Place   
32         Shopping Mall                   Gym           Pizza Place   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
39          Dessert Shop                  Café            Restaurant   
47          Dessert Shop           IT Services             Hotel Bar   
30          Dessert Shop           IT Services             Hotel Bar   
31          Dessert Shop           IT Services             Hotel Bar   
32          Dessert Shop           IT Services             Hotel Bar   

   9th Most Common Venue 10th Most Common Venue  
39           Cheese Shop    Fried Chicken Joint  
47                 Hotel   Gym / Fitness Center  
30                 Hotel   Gym / Fitness Center  
31                 Hotel   Gym / Fitness Center  
32                 Hotel   Gym / Fitness Center

In [35]:
# Create a map for visual reviews
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, place, poi, cluster in zip(pune_final['latitude'], pune_final['longitude'], pune_final['PostalCode'], pune_final['Place'], pune_final['Neighborhoods'], pune_final['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(place, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [117]:
# Analyze cluster 1
pune_final.loc[pune_final['Cluster Labels'] == 0, pune_final.columns[[1] + list(range(5, pune_final.shape[1]))]]

Place  Cluster Labels 1st Most Common Venue  \
39          Viman nagar               0     Indian Restaurant   
21     S.S.C.Exam Board               0        Ice Cream Shop   
50             Armament               0                  Café   
22  Shivajinagar (Pune)               0        Ice Cream Shop   
23              Yerwada               0     Indian Restaurant   
34         Dapodi Bazar               0             Hotel Bar   
37           Dukirkline               0     Indian Restaurant   
38        Vadgaon Sheri               0     Indian Restaurant   
33               Dapodi               0             Hotel Bar   
78                Wakad               0     Indian Restaurant   
36                9 DRD               0     Indian Restaurant   
64               Akurdi               0     Indian Restaurant   
19       Film Institute               0           Snack Place   
72               N.I.A.               0                Market   
75    Anandnagar (Pune)               0     Indian Restaurant   
76           Karvenagar               0     Indian Restaurant   
20  Congress House Road               0        Ice Cream Shop   
79                Warje               0     Indian Restaurant   
17           A.R. Shala               0           Snack Place   
77          Navsahyadri               0     Indian Restaurant   
18      Deccan Gymkhana               0           Snack Place   

   2nd Most Common Venue          3rd Most Common Venue 4th Most Common Venue  \
39           Coffee Shop                           Café        Ice Cream Shop   
21     Indian Restaurant           Fast Food Restaurant       Motorcycle Shop   
50    Chinese Restaurant                 Ice Cream Shop                   Gym   
22     Indian Restaurant           Fast Food Restaurant       Motorcycle Shop   
23         Movie Theater                Motorcycle Shop                   ATM   
34                 Hotel  Vegetarian / Vegan Restaurant               Theater   
37           Coffee Shop                           Café        Ice Cream Shop   
38           Coffee Shop                           Café        Ice Cream Shop   
33                 Hotel  Vegetarian / Vegan Restaurant               Theater   
78          Dessert Shop                            Gym                  Café   
36           Coffee Shop                           Café        Ice Cream Shop   
64           Snack Place           Fast Food Restaurant           Coffee Shop   
19     Indian Restaurant                          Hotel                Bistro   
72                   Gym              Convenience Store        Ice Cream Shop   
75          Dessert Shop                            Gym                  Café   
76          Dessert Shop                            Gym                  Café   
20     Indian Restaurant           Fast Food Restaurant       Motorcycle Shop   
79          Dessert Shop                            Gym                  Café   
17     Indian Restaurant                          Hotel                Bistro   
77          Dessert Shop                            Gym                  Café   
18     Indian Restaurant                          Hotel                Bistro   

             5th Most Common Venue 6th Most Common Venue  \
39                    Dessert Shop    Chinese Restaurant   
21                          Bakery          Burger Joint   
50                     Snack Place        Shop & Service   
22                          Bakery          Burger Joint   
23         South Indian Restaurant          Soccer Field   
34            Gym / Fitness Center                   Gym   
37                    Dessert Shop    Chinese Restaurant   
38                    Dessert Shop    Chinese Restaurant   
33            Gym / Fitness Center                   Gym   
78  Multicuisine Indian Restaurant                 Hotel   
36                    Dessert Shop    Chinese Restaurant   
64                Department Store                 Hotel   
19            Gym / Fitn

In [118]:
# Analyze cluster 2
pune_final.loc[pune_final['Cluster Labels'] == 1, pune_final.columns[[1] + list(range(5, pune_final.shape[1]))]]

Place  Cluster Labels 1st Most Common Venue  \
10              Pune City H.O               1      Department Store   
1               Dr.B.A. Chowk               1      Department Store   
2              Ghorpuri Bazar               1      Department Store   
4             Pune Cantt East               1      Department Store   
9                   Nana Peth               1      Department Store   
6              Pune New Bazar               1      Department Store   
8                Bajirao Road               1      Department Store   
5                    Pune H.O               1      Department Store   
14               H.E. Factory               1      Department Store   
15                     Khadki               1      Department Store   
3                N.W. College               1      Department Store   
11               Raviwar Peth               1      Department Store   
7             Sachapir Street               1      Department Store   
28                    Parvati               1      Department Store   
16               Khadki Bazar               1      Department Store   
29               Parvati Gaon               1      Department Store   
0                   C D A (O)               1      Department Store   
27                N.C.L. Pune               1      Department Store   
13  Ammunition Factory Khadki               1      Department Store   
26                 Baner Road               1      Department Store   
12      Shukrawar Peth (Pune)               1      Department Store   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
10               Theater             Hotel Bar                 Hotel   
1                Theater             Hotel Bar                 Hotel   
2                Theater             Hotel Bar                 Hotel   
4                Theater             Hotel Bar                 Hotel   
9                Theater             Hotel Bar                 Hotel   
6                Theater             Hotel Bar                 Hotel   
8                Theater             Hotel Bar                 Hotel   
5                Theater             Hotel Bar                 Hotel   
14                Bakery           Men's Store          Dessert Shop   
15                Bakery           Men's Store          Dessert Shop   
3                Theater             Hotel Bar                 Hotel   
11               Theater             Hotel Bar                 Hotel   
7                Theater             Hotel Bar                 Hotel   
28               Theater             Hotel Bar                 Hotel   
16                Bakery           Men's Store          Dessert Shop   
29               Theater             Hotel Bar                 Hotel   
0                Theater             Hotel Bar                 Hotel   
27               Theater             Hotel Bar                 Hotel   
13                Bakery           Men's Store          Dessert Shop   
26               Theater             Hotel Bar                 Hotel   
12               Theater             Hotel Bar                 Hotel   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
10  Gym / Fitness Center                   Gym                Garden   
1   Gym / Fitness Center                   Gym                Garden   
2   Gym / Fitness Center                   Gym                Garden   
4   Gym / Fitness Center                   Gym                Garden   
9   Gym / Fitness Center                   Gym                Garden   
6   Gym / Fitness Center                   Gym                Garden   
8   Gym / Fitness Center                   Gym                Garden   
5   Gym / Fitness Center                   Gym                Garden   
14                 Hotel  Gym / Fitness Center                   Gym   
15                 Hotel  Gym / Fitness Center                   Gym   
3   Gym / Fitness Center                   Gym                Garden   
11  Gym / Fi

In [119]:
# Analyze cluster 3
pune_final.loc[pune_final['Cluster Labels'] == 2, pune_final.columns[[1] + list(range(5, pune_final.shape[1]))]]

Place  Cluster Labels 1st Most Common Venue  \
60          Chinchwadgaon               2            Restaurant   
56   Shaniwar Peth (Pune)               2            Restaurant   
24             Aundh T.S.               2            Restaurant   
57         Airport (Pune)               2            Restaurant   
55          Sadashiv Peth               2            Restaurant   
59      Vidyanagar (Pune)               2            Restaurant   
65              Bibvewadi               2            Restaurant   
54           S.P. College               2            Restaurant   
58            Iaf Station               2            Restaurant   
53  Rashtra Bhasha Bhavan               2            Restaurant   
61               Punawale               2            Restaurant   
51          Lokmanyanagar               2            Restaurant   
63               Thergaon               2            Restaurant   
70  Ex. Serviceman Colony               2            Restaurant   
66     Market Yard (Pune)               2            Restaurant   
67         Salisbury Park               2            Restaurant   
62             Thathawade               2            Restaurant   
69         Bhusari Colony               2            Restaurant   
25            Ganeshkhind               2            Restaurant   
71                Kothrud               2            Restaurant   
52           Narayan Peth               2            Restaurant   
68             T.V. Nagar               2            Restaurant   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
60                 Hotel           Outlet Mall          Soccer Field   
56                 Hotel           Outlet Mall          Soccer Field   
24                 Hotel           Outlet Mall          Soccer Field   
57                 Hotel           Outlet Mall          Soccer Field   
55                 Hotel           Outlet Mall          Soccer Field   
59                 Hotel           Outlet Mall          Soccer Field   
65                 Hotel           Outlet Mall          Soccer Field   
54                 Hotel           Outlet Mall          Soccer Field   
58                 Hotel           Outlet Mall          Soccer Field   
53                 Hotel           Outlet Mall          Soccer Field   
61                 Hotel           Outlet Mall          Soccer Field   
51                 Hotel           Outlet Mall          Soccer Field   
63                 Hotel           Outlet Mall          Soccer Field   
70                 Hotel           Outlet Mall          Soccer Field   
66                 Hotel           Outlet Mall          Soccer Field   
67                 Hotel           Outlet Mall          Soccer Field   
62                 Hotel           Outlet Mall          Soccer Field   
69                 Hotel           Outlet Mall          Soccer Field   
25                 Hotel           Outlet Mall          Soccer Field   
71                 Hotel           Outlet Mall          Soccer Field   
52                 Hotel           Outlet Mall          Soccer Field   
68                 Hotel           Outlet Mall          Soccer Field   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
60  Gym / Fitness Center                   Gym                Garden   
56  Gym / Fitness Center                   Gym                Garden   
24  Gym / Fitness Center                   Gym                Garden   
57  Gym / Fitness Center                   Gym                Garden   
55  Gym / Fitness Center                   Gym                Garden   
59  Gym / Fitness Center                   Gym                Garden   
65  Gym / Fitness Center                   Gym                Garden   
54  Gym / Fitness Center                   Gym                Garden   
58  Gym / Fitness Center                   Gym                Garden   
53  Gym / Fitness Center                   Gym                Garden   
61  Gym / Fitness Center     

In [120]:
# Analyze cluster 4
pune_final.loc[pune_final['Cluster Labels'] == 3, pune_final.columns[[1] + list(range(5, pune_final.shape[1]))]]

Place  Cluster Labels 1st Most Common Venue  \
45            Pimpri Colony               3     Indian Restaurant   
47          Masulkar Colony               3     Indian Restaurant   
44                 Kalewadi               3     Indian Restaurant   
43  Shivaji Housing Society               3     Indian Restaurant   
48               Pimpri P F               3     Indian Restaurant   
46           Pimpri Waghire               3     Indian Restaurant   
31    Mangalwar Peth (Pune)               3     Indian Restaurant   
41        Govt. Polytechnic               3     Indian Restaurant   
30               Kasba Peth               3     Indian Restaurant   
35             Hadpsar I.E.               3     Indian Restaurant   
49           Chinchwad East               3     Indian Restaurant   
42             Model Colony               3     Indian Restaurant   
40               Dighi Camp               3         Shopping Mall   
32               Rasta Peth               3     Indian Restaurant   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
45           Snack Place         Shopping Mall           Pizza Place   
47           Snack Place         Shopping Mall           Pizza Place   
44           Snack Place         Shopping Mall           Pizza Place   
43           Snack Place         Shopping Mall           Pizza Place   
48           Snack Place         Shopping Mall           Pizza Place   
46           Snack Place         Shopping Mall           Pizza Place   
31           Snack Place         Shopping Mall           Pizza Place   
41           Snack Place         Shopping Mall           Pizza Place   
30           Snack Place         Shopping Mall           Pizza Place   
35           Snack Place         Shopping Mall           Pizza Place   
49           Snack Place         Shopping Mall           Pizza Place   
42           Snack Place         Shopping Mall           Pizza Place   
40           Pizza Place                Bakery                  Café   
32           Snack Place         Shopping Mall           Pizza Place   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
45          Burger Joint      Department Store  Gym / Fitness Center   
47          Burger Joint      Department Store  Gym / Fitness Center   
44          Burger Joint      Department Store  Gym / Fitness Center   
43          Burger Joint      Department Store  Gym / Fitness Center   
48          Burger Joint      Department Store  Gym / Fitness Center   
46          Burger Joint      Department Store  Gym / Fitness Center   
31          Burger Joint      Department Store  Gym / Fitness Center   
41          Burger Joint      Department Store  Gym / Fitness Center   
30          Burger Joint      Department Store  Gym / Fitness Center   
35          Burger Joint      Department Store  Gym / Fitness Center   
49          Burger Joint      Department Store  Gym / Fitness Center   
42          Burger Joint      Department Store  Gym / Fitness Center   
40      Department Store                 Hotel  Gym / Fitness Center   
32          Burger Joint      Department Store  Gym / Fitness Center   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  \
45                   Gym                Garden    Fried Chicken Joint   
47                   Gym                Garden    Fried Chicken Joint   
44                   Gym                Garden    Fried Chicken Joint   
43                   Gym                Garden    Fried Chicken Joint   
48                   Gym                Garden    Fried Chicken Joint   
46                   Gym                Garden    Fried Chicken Joint   
31                   Gym                Garden    Fried Chicken Joint   
41                   Gym                Garden    Fried Chicken Joint   
30                   Gym                Garden    Fried Chicken Joint   
35                   Gym                Garden    Fried Chicken Joint   
49                   

In [36]:
# Analyze cluster 5
pune_final.loc[pune_final['Cluster Labels'] == 4, pune_final.columns[[1] + list(range(5, pune_final.shape[1]))]]

Place  Cluster Labels 1st Most Common Venue  \
73        Lohogaon               4     Mobile Phone Shop   
33          Dapodi               4             Hotel Bar   
74  Vadgaon Shinde               4     Mobile Phone Shop   
34    Dapodi Bazar               4             Hotel Bar   

            2nd Most Common Venue 3rd Most Common Venue  \
73  Vegetarian / Vegan Restaurant     Electronics Store   
33                          Hotel     Mobile Phone Shop   
74  Vegetarian / Vegan Restaurant     Electronics Store   
34                          Hotel     Mobile Phone Shop   

            4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
73                    IT Services             Hotel Bar                 Hotel   
33  Vegetarian / Vegan Restaurant     Electronics Store           IT Services   
74                    IT Services             Hotel Bar                 Hotel   
34  Vegetarian / Vegan Restaurant     Electronics Store           IT Services   

   7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
73  Gym / Fitness Center                   Gym                Garden   
33  Gym / Fitness Center                   Gym                Garden   
74  Gym / Fitness Center                   Gym                Garden   
34  Gym / Fitness Center                   Gym                Garden   

   10th Most Common Venue  
73    Fried Chicken Joint  
33    Fried Chicken Joint  
74    Fried Chicken Joint  
34    Fried Chicken Joint